In [ ]:
#!pip install beautifulsoup4 requests selenium

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import re

Collecting Signal IDs

In [ ]:
# The base URL of the website we want to crawl
base_url = 'https://www.mql5.com/en/signals/mt5/list/page'

# The list of pages we want to crawl // Useless code 
page_urls = ['1']

# The CSV file we want to write the data to
csv_file = open('signal_list.csv', 'w', newline='')
writer = csv.writer(csv_file)
writer.writerow(['Signal Id'])

# Loop over each page URL and extract the Signal ID data
for page_url in page_urls:
    # Construct the full URL for the page
    url = base_url + page_url

    # Make a GET request to the page
    response = requests.get(url)

    # Parse the HTML content of the page using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all the element has id 
    ids = soup.find_all('div', class_='button button_white-and-green')

    #extrac the id
    for id in ids:
        name = id.get("data-id")
        # Write the data to the CSV file
        writer.writerow([name])

# Close the CSV file
csv_file.close()


Viewing IDs List

In [3]:
IDs = pd.read_csv("signal_list.csv")
IDs.head


<bound method NDFrame.head of     Signal Id
0     1368342
1     1636506
2      843451
3     1486477
4     1818780
5     1598419
6     1245170
7     1722050
8     1725375
9     1334849
10    1712504
11    1857419
12    1788678
13    1766488
14    1726461
15    1829142
16    1895985
17     951062
18    1778541
19    1888377
20    1405177
21    1759554
22    1694912
23    1184516
24    1855883
25    1273204
26    1894325
27    1780040
28    1531887
29     910374
30    1526229
31    1312512
32    1612455
33    1274869
34    1594934
35    1908576
36    1507561
37    1607553
38    1415131
39    1882495
40    1059619
41    1196985
42    1500355
43    1722761
44    1279728
45    1729929
46    1833703
47    1815355>

In [4]:
base_url_singleID = 'https://www.mql5.com/en/signals/'

#load the csv containing Signal IDs
with open('signal_details.csv', 'w', newline='') as signal_detailsFile:
  writer = csv.writer(signal_detailsFile)
  writer.writerow(['Signal Id','Trades','ProfitTrades','LossTrades','BestTrades','WorstTrades','GrossProfit','GrossLoss','SharpeRatio','RecoveryFactor','ProfitFactor','ExpectedPayoff'])
  for row in IDs['Signal Id']:
    url = base_url_singleID + str(row) + "?source=Unknown#!tab=stats"
    
    #Crawl each signal
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    values = soup.find_all('div', class_='s-data-columns__value')

    # Loop over each product and extract the data
    signalTrades = values[0].text 


    # Get total number of profit trades
    s = values[1].text
    number_regex = r"([\d\s]+)\s*\(.*\)"
    match = re.search(number_regex, s)
    if match:
        number = match.group(1)
        # Remove spaces and normalize the number
        normalized_number = float(number.replace(" ", "").replace(",", ""))
    signalProfitTrades = normalized_number

    # Get total number of loss trades

    s = values[2].text
    number_regex = r"([\d\s]+)\s*\(.*\)"
    match = re.search(number_regex, s)
    if match:
        number = match.group(1)
        # Remove spaces and normalize the number
        normalized_number = float(number.replace(" ", "").replace(",", ""))
    signalLossTrades = normalized_number

    signalBestTrade = values[3].find('i').text
    signalWorstTrade = values[4].find('i').text

    signalGrossProfit = values[5].find('i').text
    signalGrossLoss = values[6].find('i').text

    signalSharpeRatio = values[9].text

    signalRecoveryFactor = values[15].text


    signalProfitFactor = values[18].text

    signalExpectedPayoff = values[19].find('i').text
    writer.writerow([row,signalTrades,signalProfitTrades,signalLossTrades,signalBestTrade,signalWorstTrade,signalGrossProfit,signalGrossLoss,signalSharpeRatio,signalRecoveryFactor,signalProfitFactor,signalExpectedPayoff])
    




View Signal Information

In [ ]:
signals = pd.read_csv("signal_details.csv")
signals